In [3]:
import getpass
import os

from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
llm = init_chat_model("openai:gpt-5")

Enter API key for OpenAI:  ········


In [23]:
from langchain_core.tools import tool

from typing import Annotated
import subprocess

@tool
def compile_c(code: Annotated[str, "ANSI-C source code"]):
    """
    Compile C source code, returning the error message
    or 'success' if the program compiled successfully.
    """

    try:
        subprocess.run(["gcc", "-x", "c", "-"], input=code.encode(), capture_output=True, check=True)
        return "Compilation succeeded!"
    except subprocess.CalledProcessError as e:
        return f"Compilation failed!\n\n{e.stderr.decode()}"

In [27]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="openai:gpt-5",
    tools=[compile_c],
    prompt="You are an expert C programmer."
)

agent.invoke({
    "messages": ["Write a C program that implements merge sort"]
})

{'messages': [HumanMessage(content='Write a C program that implements merge sort', additional_kwargs={}, response_metadata={}, id='e993239e-d4a5-42c0-8e18-245d01e974c5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uCsmz9rBVipFUOWJ8gWKS2wf', 'function': {'arguments': '{"code":"#include <stdio.h>\\n#include <stdlib.h>\\n\\nstatic void merge(int *a, int left, int mid, int right, int *tmp) {\\n    int i = left;\\n    int j = mid + 1;\\n    int k = left;\\n\\n    while (i <= mid && j <= right) {\\n        if (a[i] <= a[j]) {\\n            tmp[k++] = a[i++];\\n        } else {\\n            tmp[k++] = a[j++];\\n        }\\n    }\\n    while (i <= mid) tmp[k++] = a[i++];\\n    while (j <= right) tmp[k++] = a[j++];\\n\\n    for (i = left; i <= right; ++i) {\\n        a[i] = tmp[i];\\n    }\\n}\\n\\nstatic void merge_sort_rec(int *a, int left, int right, int *tmp) {\\n    if (left >= right) return;\\n    int mid = left + (right - left) / 2;\\n    merge_sort_rec(a, le

In [46]:
import tempfile

@tool
def compile_latex(source: Annotated[str, "LaTeX source code"]) -> str:
    """
    Compile LaTeX source code in an environment with
    all texlive packages. Returns any errors produced,
    and whether the compilation succeeds.
    """

    with tempfile.TemporaryDirectory() as working_dir:
        source_path = os.path.join(working_dir, "input.tex")
        with open(source_path, "w") as source_file:
            source_file.write(source)

        output_name = "output"

        try:
            subprocess.run(
                [
                    "pdflatex",
                    f"-jobname={output_name}",
                    "-interaction=nonstopmode",
                    source_path,
                ],
                cwd=working_dir,
                capture_output = True,
                check = True
            )
            return "Compilation succeeded!"
        except subprocess.CalledProcessError as e:
            return f"Compilation failed!\n\n{e.stdout.decode()}\n\n{e.stderr.decode()}"

agent = create_react_agent(
    model="openai:gpt-5",
    tools=[compile_latex],
    prompt="You are an expert LaTeX programmer."
)

agent.invoke({
    "messages": "Create a set of lecture slides with LaTeX beamer that teach the A* algorithm to second-year computer science students in a Data Structures and Algorithms course."
})

{'messages': [HumanMessage(content='Create a set of lecture slides with LaTeX beamer that teach the A* algorithm to second-year computer science students in a Data Structures and Algorithms course.', additional_kwargs={}, response_metadata={}, id='8a645519-1026-4a27-83d2-528162eb94b4'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eGwm2tn5FbrXKig4TeoslK1b', 'function': {'arguments': '{"source":"% !TeX program = pdflatex\\n\\\\documentclass[aspectratio=169]{beamer}\\n\\n% Theme and appearance\\n\\\\usetheme{Madrid}\\n\\\\setbeamertemplate{footline}[frame number]\\n\\\\setbeamertemplate{navigation symbols}{}\\n\\n% Packages\\n\\\\usepackage[utf8]{inputenc}\\n\\\\usepackage[T1]{fontenc}\\n\\\\usepackage{amsmath, amssymb}\\n\\\\usepackage{algorithm2e}\\n\\\\usepackage{tikz}\\n\\\\usetikzlibrary{arrows.meta, positioning}\\n\\n% Title info\\n\\\\title[A* Algorithm]{A* Algorithm: Informed Search for Optimal Paths}\\n\\\\author{Instructor: Your Name}\\n\\\\institute{D